In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [12]:
import spacy
import gensim
from gensim.models import TfidfModel

In [1]:
data=["i love my mum","i love my mum","i love my mum","my dad in better shape","i love my dad and my dad","i love my mum and dad"]

In [31]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV","PROPN","PART"]):
    #nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    nlp = spacy.load("en_core_web_sm")
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [5]:
import json
import funzioni_preprocessing_text as fpt

In [8]:
temp=json.load(open("allDocuments2.json","r"))

In [9]:
temp[0]

"The fall of Roe v. Wade has pushed the U.S. into turbulence that feels all too familiar to those who lived through the pre-Roe era. We spoke to women who remember a time without Roe — and the early years after the 1973 ruling.\nCatherine Starr told The New York Times in an interview that she had become pregnant when she was 16, and, without the option of a safe, legal abortion, she gave her baby up for adoption. She was 17 when she attended her first demonstration to push for abortion rights outside City Hall in St. Louis. It was May 24, 1973, just months after the Supreme Court ruled on Roe v. Wade. Starr went to the rally because she “wanted to be able to make sure that the next little girl that gets pregnant has an option,” she said.\nThe same year that Roe was decided, our photographer William E. Sauro documented scenes inside Women's Medical Services (second and fourth photo), an abortion clinic on Walnut Street in downtown Philadelphia, and our photographer Librado Romero took p

In [29]:
prova="i'm a new to not good samaritan don't you think covid-19 is a good thing"

In [ ]:
["NOUN", "ADJ", "VERB", "ADV","PROPN","PART"]

In [33]:
a=lemmatization([prova])


PRON I i
AUX be 'm
DET a a
ADJ new new
ADP to to
PART not not
ADJ good good
PROPN samaritan samaritan
AUX do do
PART not n't
PRON you you
VERB think think
PROPN covid-19 covid-19
AUX be is
DET a a
ADJ good good
NOUN thing thing


In [34]:
a

['new not good samaritan not think covid-19 good thing']

In [35]:
gen_words(a)

[['new', 'not', 'good', 'samaritan', 'not', 'think', 'covid', 'good', 'thing']]

In [23]:
print(fpt.clear_corpus([prova])[0])

[['m', 'new', 'tw', 'not_good', 'samaritan', 'don', 't', 'think', 'covid', 'good', 'thing']]


In [5]:
lemmatized_texts = lemmatization(data)
data_words = gen_words(lemmatized_texts)

In [22]:
lemmatized_texts

['love mum',
 'love mum',
 'love mum',
 'dad well shape',
 'love dad dad',
 'love mum dad']

In [23]:
data_words

[['love', 'mum'],
 ['love', 'mum'],
 ['love', 'mum'],
 ['dad', 'well', 'shape'],
 ['love', 'dad', 'dad'],
 ['love', 'mum', 'dad']]

In [6]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram = gensim.models.phrases.Phraser(bigram_phrases)
data_bigrams = ([bigram[doc] for doc in data_words])

trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)
trigram = gensim.models.phrases.Phraser(trigram_phrases)
data_bigrams_trigrams = ([trigram[bigram[doc]] for doc in data_bigrams])

In [24]:
data_bigrams

[['love', 'mum'],
 ['love', 'mum'],
 ['love', 'mum'],
 ['dad', 'well', 'shape'],
 ['love', 'dad', 'dad'],
 ['love', 'mum', 'dad']]

In [25]:
data_bigrams_trigrams

[['love', 'mum'],
 ['love', 'mum'],
 ['love', 'mum'],
 ['dad', 'well', 'shape'],
 ['love', 'dad', 'dad'],
 ['love', 'mum', 'dad']]

In [9]:
id2word = gensim.corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [32]:
id2word2 = gensim.corpora.Dictionary(data_bigrams_trigrams)

In [33]:
corpus2 = [id2word.doc2bow(text) for text in texts]

In [35]:
tfidf2 = TfidfModel(corpus2, id2word=id2word2)

In [38]:
for i in range(len(corpus2)):
    print(tfidf2[corpus2[i]])

[(0, 0.410107005098448), (1, 0.9120374139086519)]
[(0, 0.410107005098448), (1, 0.9120374139086519)]
[(0, 0.410107005098448), (1, 0.9120374139086519)]
[(2, 0.26385258936279504), (3, 0.682049049221003), (4, 0.682049049221003)]
[(0, 0.13039433685974838), (2, 0.9914622115415728)]
[(0, 0.22140753228371998), (1, 0.49238845143712107), (2, 0.8417436174628159)]


In [39]:
for i in range(len(corpus2)):
    print(corpus2[i])

[(0, 1), (1, 1)]
[(0, 1), (1, 1)]
[(0, 1), (1, 1)]
[(2, 1), (3, 1), (4, 1)]
[(0, 1), (2, 2)]
[(0, 1), (1, 1), (2, 1)]


In [55]:
import copy

In [56]:
id2word3 = gensim.corpora.Dictionary(data_bigrams_trigrams)

corpus3 = [id2word3.doc2bow(doc) for doc in data_bigrams_trigrams]
tfidf3 = TfidfModel(corpus, id2word=id2word3)

low_value = 0.2
low_value_words = []
for bow in corpus3:
    low_value_words += [id for id, value in tfidf3[bow] if value < low_value]
id2word3_2=copy.copy(id2word3)
id2word3_2.filter_tokens(low_value_words)
new_corpus3 = [id2word3_2.doc2bow(doc) for doc in data_bigrams_trigrams]

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [20]:
print(corpus)

[[(0, 1), (1, 1)], [(0, 1), (1, 1)], [(0, 1), (1, 1)], [(2, 1), (3, 1), (4, 1)], [(0, 1), (2, 2)], [(0, 1), (1, 1), (2, 1)]]


In [21]:
test_doc = corpus[-1]
result = lda_model.get_document_topics(test_doc)
idxMax=max(result, key=lambda x: x[1])[0]
lda_model.show_topic(idxMax)

[('love', 0.40297326),
 ('mum', 0.32395872),
 ('dad', 0.25726476),
 ('well', 0.0079016),
 ('shape', 0.007901599)]

In [18]:
result

[(0, 0.012870085),
 (1, 0.0128699625),
 (2, 0.026503067),
 (3, 0.012870058),
 (4, 0.012869979),
 (5, 0.012869807),
 (6, 0.012870204),
 (7, 0.012870204),
 (8, 0.8705361),
 (9, 0.012870537)]

In [17]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.045874324), (1, 0.035836946), (2, 0.4376528), (5, 0.15934138), (7, 0.057556253), (8, 0.07231899), (9, 0.19118835)]
[(2, 0.4376528), (9, 0.19118835), (5, 0.15934138), (8, 0.07231899), (7, 0.057556253), (0, 0.045874324), (1, 0.035836946)]


In [20]:
lda_model.save("models/test_model.model")

In [22]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [23]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.045875695), (1, 0.035836957), (2, 0.43765357), (5, 0.15934144), (7, 0.057556026), (8, 0.072318755), (9, 0.1911866)]
[(2, 0.43765357), (9, 0.1911866), (5, 0.15934144), (8, 0.072318755), (7, 0.057556026), (0, 0.045875695), (1, 0.035836957)]


## Vizualizing the Data

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
# vis